<a href="https://colab.research.google.com/github/ffelfis/OrgaDatosTPs/blob/main/TP2/nbFacu_Random_Forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lectura de datos de Google Drive

In [1]:
# Lectura de Dataset desde Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Carga de librerías y directorios

In [2]:
import pandas as pd
import numpy as np

# Rutas de los archivos a usar

In [3]:
# Ruta train_values.csv
dir_values = '/content/drive/My Drive/75.06 - Organización de Datos/TP1/Data/train_values.csv'

# Ruta train_labels.csv
dir_labels = '/content/drive/My Drive/75.06 - Organización de Datos/TP1/Data/train_labels.csv'

# Ruta test_values.csv
dir_test = '/content/drive/My Drive/75.06 - Organización de Datos/TP1/Data/test_values.csv'

# Ruta submission_format.csv
dir_sub_format = '/content/drive/My Drive/75.06 - Organización de Datos/TP1/Data/submission_format.csv'

# Ruta de Binary Encodings para train_values.csv
dir_Binary_train = '/content/drive/My Drive/75.06 - Organización de Datos/TP2/Binary Encodings/train'

# Ruta de Binary Encodings para test_values.csv
dir_Binary_test = '/content/drive/My Drive/75.06 - Organización de Datos/TP2/Binary Encodings/test'

---
#Entrenamiento
---
### Carga de train

In [4]:
columnas = ['building_id',
 'geo_level_1_id',
 'geo_level_2_id',
 'geo_level_3_id',
 'count_floors_pre_eq',
 'age',
 'area_percentage',
 'height_percentage',
 'has_superstructure_adobe_mud',
 'has_superstructure_mud_mortar_stone',
 'has_superstructure_stone_flag',
 'has_superstructure_cement_mortar_stone',
 'has_superstructure_mud_mortar_brick',
 'has_superstructure_cement_mortar_brick',
 'has_superstructure_timber',
 'has_superstructure_bamboo',
 'has_superstructure_rc_non_engineered',
 'has_superstructure_rc_engineered',
 'has_superstructure_other',
 'count_families',
 'has_secondary_use',
 'has_secondary_use_agriculture',
 'has_secondary_use_hotel',
 'has_secondary_use_rental',
 'has_secondary_use_institution',
 'has_secondary_use_school',
 'has_secondary_use_industry',
 'has_secondary_use_health_post',
 'has_secondary_use_gov_office',
 'has_secondary_use_use_police',
 'has_secondary_use_other']

In [5]:
# Carga de train_values.csv
train = pd.read_csv(dir_values, usecols=columnas)
# Carga de train_labels.csv
labels = pd.read_csv(dir_labels)

### Cambio de tipos de datos

In [6]:
memoria = train.memory_usage(deep=True).sum()/(2**20)
print('Se usan', np.round(memoria, 2), 'MiB')

Se usan 61.64 MiB


In [7]:
# Columnas numéricas
train['building_id'] = train['building_id'].astype('int32')
train['geo_level_1_id'] = train['geo_level_1_id'].astype('int8')
train['geo_level_2_id'] = train['geo_level_2_id'].astype('int16')
train['geo_level_3_id'] = train['geo_level_3_id'].astype('int16')
train['count_floors_pre_eq'] = train['count_floors_pre_eq'].astype('int8')
train['age'] = train['age'].astype('int16')
train['area_percentage'] = train['area_percentage'].astype('int8')
train['height_percentage'] = train['height_percentage'].astype('int8')
train['count_families'] = train['count_families'].astype('int8')

# Columnas Booleanas
train['has_superstructure_adobe_mud'] = train['has_superstructure_adobe_mud'].astype('bool')
train['has_superstructure_mud_mortar_stone'] = train['has_superstructure_mud_mortar_stone'].astype('bool')
train['has_superstructure_stone_flag'] = train['has_superstructure_stone_flag'].astype('bool')
train['has_superstructure_cement_mortar_stone'] = train['has_superstructure_cement_mortar_stone'].astype('bool')
train['has_superstructure_mud_mortar_brick'] = train['has_superstructure_mud_mortar_brick'].astype('bool')
train['has_superstructure_cement_mortar_brick'] = train['has_superstructure_cement_mortar_brick'].astype('bool')
train['has_superstructure_timber'] = train['has_superstructure_timber'].astype('bool')
train['has_superstructure_bamboo'] = train['has_superstructure_bamboo'].astype('bool')
train['has_superstructure_rc_non_engineered'] = train['has_superstructure_rc_non_engineered'].astype('bool')
train['has_superstructure_rc_engineered'] = train['has_superstructure_rc_engineered'].astype('bool')
train['has_superstructure_other'] = train['has_superstructure_other'].astype('bool')
train['has_secondary_use'] = train['has_secondary_use'].astype('bool')
train['has_secondary_use_agriculture'] = train['has_secondary_use_agriculture'].astype('bool')
train['has_secondary_use_hotel'] = train['has_secondary_use_hotel'].astype('bool')
train['has_secondary_use_rental'] = train['has_secondary_use_rental'].astype('bool')
train['has_secondary_use_institution'] = train['has_secondary_use_institution'].astype('bool')
train['has_secondary_use_school'] = train['has_secondary_use_school'].astype('bool')
train['has_secondary_use_industry'] = train['has_secondary_use_industry'].astype('bool')
train['has_secondary_use_health_post'] = train['has_secondary_use_health_post'].astype('bool')
train['has_secondary_use_gov_office'] = train['has_secondary_use_gov_office'].astype('bool')
train['has_secondary_use_use_police'] = train['has_secondary_use_use_police'].astype('bool')
train['has_secondary_use_other'] = train['has_secondary_use_other'].astype('bool')

In [8]:
memoria = train.memory_usage(deep=True).sum()/(2**20)
print('Se usan', np.round(memoria, 2), 'MiB')

Se usan 9.2 MiB


### Carga de columnas codificadas: Binary Encoding

In [9]:
# 4 columnas
foundation_type = pd.read_csv(dir_Binary_train+f'/BE_foundation_type.csv', dtype='uint8')
train = train.join(foundation_type)

In [10]:
# 4 columnas
ground_floor_type = pd.read_csv(dir_Binary_train+f'/BE_ground_floor_type.csv', dtype=('uint8'))
train = train.join(ground_floor_type)

In [11]:
# 3 columnas
land_surface_condition = pd.read_csv(dir_Binary_train+f'/BE_land_surface_condition.csv', dtype='uint8')
train = train.join(land_surface_condition)

In [12]:
# 3 columnas
legal_ownership_status = pd.read_csv(dir_Binary_train+f'/BE_legal_ownership_status.csv', dtype='uint8')
train = train.join(legal_ownership_status)

In [13]:
# 3 columnas
other_floor_type = pd.read_csv(dir_Binary_train+f'/BE_other_floor_type.csv', dtype='uint8')
train = train.join(other_floor_type)

In [14]:
# 5 columnas
plan_configuration = pd.read_csv(dir_Binary_train+f'/BE_plan_configuration.csv', dtype='uint8')
train = train.join(plan_configuration)

In [15]:
# 3 columnas
position = pd.read_csv(dir_Binary_train+f'/BE_position.csv', dtype='uint8')
train = train.join(position)

In [16]:
# 3 columnas
roof_type = pd.read_csv(dir_Binary_train+f'/BE_roof_type.csv', dtype='uint8')
train = train.join(roof_type)

In [17]:
memoria = train.memory_usage(deep=True).sum()/(2**20)
print('Se usan', np.round(memoria, 2), 'MiB')

Se usan 16.15 MiB


### Labels

In [18]:
memoria = labels.memory_usage(deep=True).sum()/(2**20)
print('Se usan', np.round(memoria, 2), 'MiB')

Se usan 3.98 MiB


In [19]:
labels['building_id'] = labels['building_id'].astype('int32')
labels['damage_grade'] = labels['damage_grade'].astype('int8')

In [20]:
memoria = labels.memory_usage(deep=True).sum()/(2**20)
print('Se usan', np.round(memoria, 2), 'MiB')

Se usan 1.24 MiB


# Modelo Random Forest Classifier

In [21]:
# Modelo
from sklearn.ensemble import RandomForestClassifier
# Para dividir el set de entrenamiento
from sklearn.model_selection import train_test_split

### División de datos para entrenamiento y objetivo

In [22]:
# Set de entrenamiento
X = train
# Labels -- solo la columna de labels
y = labels['damage_grade']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
print('X_train.shape\t', X_train.shape, '\ty_train.shape\t', y_train.shape)
print('X_test.shape\t', X_test.shape, '\ty_test.shape\t', y_test.shape)

X_train.shape	 (195450, 59) 	y_train.shape	 (195450,)
X_test.shape	 (65151, 59) 	y_test.shape	 (65151,)


### Instanciación de predictor (random_state=42)

In [23]:
# n_estimators
num_estimadores = 40
# n_jobs
workers = -1
# Instancia para predictor de Random Forest
rfc = RandomForestClassifier(n_estimators=num_estimadores, n_jobs=workers, random_state=42)

### Entrenamiento único

In [24]:
%%time
rfc.fit(X_train, y_train)

CPU times: user 35.3 s, sys: 488 ms, total: 35.8 s
Wall time: 18.9 s


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=40, n_jobs=-1,
                       oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [25]:
# Métrica de evaluación del modelo
from sklearn.metrics import accuracy_score

In [26]:
# Ahora se usa la división que faltba del train: X_test
prediction_t = rfc.predict(X_test)
print('Predicción', np.round(accuracy_score(y_test, prediction_t),4))

Predicción 0.7116


# Evaluación del predictor

In [27]:
# Para validar
from sklearn.model_selection import KFold
# Para ver progreso inline: limpia la celda en cada iteración
from IPython.display import clear_output

## División de datos para entrenamiento y objetivo

In [28]:
# Set de entrenamiento
X = train
# Labels -- solo la columna de labels
y = labels['damage_grade']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
print('X_train.shape\t', X_train.shape, '\ty_train.shape\t', y_train.shape)
print('X_test.shape\t', X_test.shape, '\ty_test.shape\t', y_test.shape)

X_train.shape	 (195450, 59) 	y_train.shape	 (195450,)
X_test.shape	 (65151, 59) 	y_test.shape	 (65151,)


## Instancias

In [29]:
# n_estimators
num_estimadores = 40
# n_jobs
workers = -1
# Instancia para predictor de Random Forest
rfc = RandomForestClassifier(n_estimators=num_estimadores, n_jobs=workers)

# n_splits
splits = 10
# Instancia para KFold
kf = KFold(n_splits=splits)

# Listas para guardar los puntajes y errores de la evaluación
accuracy = []

In [30]:
%%time
i = 1

for train_index, test_index in kf.split(train) :
    clear_output(wait=True)
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    print('Entrenando fold', i)
    rfc.fit(X_train, y_train)
    prediction_t = rfc.predict(X_test)

    accuracy.append(accuracy_score(y_test, prediction_t))
    i = i + 1

Entrenando fold 10
CPU times: user 8min 3s, sys: 3.73 s, total: 8min 7s
Wall time: 4min 16s


In [31]:
precision = np.array(accuracy)

print('Promedio precision', precision.mean().round(4), \
      '\tcon desvío estándar de', precision.std().round(4))

Promedio precision 0.7149 	con desvío estándar de 0.0023
Promedio scores	 0.7149 		con desvío estándar de 0.0023


---
# Predicción del Test
---
### Carga Test set

In [32]:
test = pd.read_csv(dir_test, usecols=columnas)

### Cambio de tipos de datos

In [35]:
memoria = test.memory_usage(deep=True).sum()/(2**20)
print('Se usan', np.round(memoria, 2), 'MiB')

Se usan 20.55 MiB


In [36]:
# Columnas numéricas
test['building_id'] = test['building_id'].astype('int32')
test['geo_level_1_id'] = test['geo_level_1_id'].astype('int8')
test['geo_level_2_id'] = test['geo_level_2_id'].astype('int16')
test['geo_level_3_id'] = test['geo_level_3_id'].astype('int16')
test['count_floors_pre_eq'] = test['count_floors_pre_eq'].astype('int8')
test['age'] = test['age'].astype('int16')
test['area_percentage'] = test['area_percentage'].astype('int8')
test['height_percentage'] = test['height_percentage'].astype('int8')
test['count_families'] = test['count_families'].astype('int8')

# Columnas Booleanas
test['has_superstructure_adobe_mud'] = test['has_superstructure_adobe_mud'].astype('bool')
test['has_superstructure_mud_mortar_stone'] = test['has_superstructure_mud_mortar_stone'].astype('bool')
test['has_superstructure_stone_flag'] = test['has_superstructure_stone_flag'].astype('bool')
test['has_superstructure_cement_mortar_stone'] = test['has_superstructure_cement_mortar_stone'].astype('bool')
test['has_superstructure_mud_mortar_brick'] = test['has_superstructure_mud_mortar_brick'].astype('bool')
test['has_superstructure_cement_mortar_brick'] = test['has_superstructure_cement_mortar_brick'].astype('bool')
test['has_superstructure_timber'] = test['has_superstructure_timber'].astype('bool')
test['has_superstructure_bamboo'] = test['has_superstructure_bamboo'].astype('bool')
test['has_superstructure_rc_non_engineered'] = test['has_superstructure_rc_non_engineered'].astype('bool')
test['has_superstructure_rc_engineered'] = test['has_superstructure_rc_engineered'].astype('bool')
test['has_superstructure_other'] = test['has_superstructure_other'].astype('bool')
test['has_secondary_use'] = test['has_secondary_use'].astype('bool')
test['has_secondary_use_agriculture'] = test['has_secondary_use_agriculture'].astype('bool')
test['has_secondary_use_hotel'] = test['has_secondary_use_hotel'].astype('bool')
test['has_secondary_use_rental'] = test['has_secondary_use_rental'].astype('bool')
test['has_secondary_use_institution'] = test['has_secondary_use_institution'].astype('bool')
test['has_secondary_use_school'] = test['has_secondary_use_school'].astype('bool')
test['has_secondary_use_industry'] = test['has_secondary_use_industry'].astype('bool')
test['has_secondary_use_health_post'] = test['has_secondary_use_health_post'].astype('bool')
test['has_secondary_use_gov_office'] = test['has_secondary_use_gov_office'].astype('bool')
test['has_secondary_use_use_police'] = test['has_secondary_use_use_police'].astype('bool')
test['has_secondary_use_other'] = test['has_secondary_use_other'].astype('bool')

In [37]:
memoria = test.memory_usage(deep=True).sum()/(2**20)
print('Se usan', np.round(memoria, 2), 'MiB')

Se usan 3.07 MiB


### Carga de columnas codificadas: Binary Encoding

In [38]:
# 4 columnas
foundation_type = pd.read_csv(dir_Binary_test+f'/BE_foundation_type.csv', dtype='uint8')
test = test.join(foundation_type)

In [39]:
# 4 columnas
ground_floor_type = pd.read_csv(dir_Binary_test+f'/BE_ground_floor_type.csv', dtype=('uint8'))
test = test.join(ground_floor_type)

In [40]:
# 3 columnas
land_surface_condition = pd.read_csv(dir_Binary_test+f'/BE_land_surface_condition.csv', dtype='uint8')
test = test.join(land_surface_condition)

In [41]:
# 3 columnas
legal_ownership_status = pd.read_csv(dir_Binary_test+f'/BE_legal_ownership_status.csv', dtype='uint8')
test = test.join(legal_ownership_status)

In [42]:
# 3 columnas
other_floor_type = pd.read_csv(dir_Binary_test+f'/BE_other_floor_type.csv', dtype='uint8')
test = test.join(other_floor_type)

In [43]:
# 5 columnas
plan_configuration = pd.read_csv(dir_Binary_test+f'/BE_plan_configuration.csv', dtype='uint8')
test = test.join(plan_configuration)

In [44]:
# 3 columnas
position = pd.read_csv(dir_Binary_test+f'/BE_position.csv', dtype='uint8')
test = test.join(position)

In [45]:
# 3 columnas
roof_type = pd.read_csv(dir_Binary_test+f'/BE_roof_type.csv', dtype='uint8')
test = test.join(roof_type)

In [46]:
memoria = test.memory_usage(deep=True).sum()/(2**20)
print('Se usan', np.round(memoria, 2), 'MiB')

Se usan 5.38 MiB


# Preparando la predicción

In [47]:
# n_estimators
num_estimadores = 200
# n_jobs
workers = -1
# Instancia para predictor de Random Forest
rfc = RandomForestClassifier(n_estimators=num_estimadores, n_jobs=workers, random_state=42)

In [49]:
%%time
rfc.fit(X, y)

CPU times: user 4min 23s, sys: 4.57 s, total: 4min 27s
Wall time: 2min 17s


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=-1, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

### Predicción

In [50]:
prediction = rfc.predict(test)

In [51]:
prediction

array([3, 2, 2, ..., 2, 2, 2], dtype=int8)

### Submission

In [56]:
sub_example = pd.read_csv(dir_sub_format)

In [62]:
sub_example

,building_id,damage_grade
0,300051,1
1,99355,1
2,890251,1
3,745817,1
4,421793,1
...,...,...
86863,310028,1
86864,663567,1
86865,1049160,1
86866,442785,1


In [58]:
submit = pd.DataFrame()
submit['building_id'] = test['building_id']
submit['damage_grade'] = prediction

In [59]:
submit

,building_id,damage_grade
0,300051,3
1,99355,2
2,890251,2
3,745817,1
4,421793,3
...,...,...
86863,310028,2
86864,663567,2
86865,1049160,2
86866,442785,2


In [66]:
submit.to_csv('submission.csv', index=False)

In [67]:
aux = pd.read_csv('submission.csv')

In [68]:
aux

,building_id,damage_grade
0,300051,3
1,99355,2
2,890251,2
3,745817,1
4,421793,3
...,...,...
86863,310028,2
86864,663567,2
86865,1049160,2
86866,442785,2
